In [1]:
import pandas as pd
import numpy as np
import csv
import h5py
import pickle
import random 
random.seed(42)
import ast
import networkx as nx
from networkx.algorithms.approximation import steiner_tree
from tqdm import tqdm
from itertools import combinations

In [2]:
ppi = h5py.File('/ix/djishnu/Alisa/Tfh/Network_analysis/data/HomoSapiens_binary_co_complex_Feb2023_1_ppr_0.4.h5', 'r')
#list(ppi.keys())
hint_data = ppi['edges']

### Make a dataframe with HINT interactions and the Gene IDs
def make_hint_df(hint_data):
    gene_sets = []
    for entry in hint_data:
        prot_1 = entry[0].decode('utf-8')
        prot_2 = entry[1].decode('utf-8')
        
        gene_sets.append([prot_1, prot_2])
    return gene_sets


hint = make_hint_df(hint_data)
hint_df = pd.DataFrame(hint)

hint_df.columns = ["GeneA", "GeneB"]

In [3]:
#sif_df = pd.read_csv("/ix/djishnu/Alisa/Tfh/ForPaper/other_figures/vinuesa_steiner_network.sif", sep="\t| ", engine="python", header=None)

sif_df = pd.read_csv("/ix/djishnu/Alisa/Tfh/ForPaper/other_figures/extended_vinuesa_hint_cxcr5_centric.sif", sep="\t| ", engine="python", header=None)
sif_df.columns = ["Source", "Interaction", "Target"]

In [48]:
# Define your protein of interest
protein = "GNAI2"

# Find all interactors: rows where VIRMA is in either column
interactors = hint_df[
    (hint_df["GeneA"] == protein) | (hint_df["GeneB"] == protein)
]

# Flatten to get all partner genes
interactors_set = set()

for _, row in interactors.iterrows():
    if row["GeneA"] == protein:
        interactors_set.add(row["GeneB"])
    else:
        interactors_set.add(row["GeneA"])

# Convert to DataFrame
interactors_df = pd.DataFrame({"Interactor": list(interactors_set)})

# Optional: add a column for the source protein
interactors_df["Source"] = protein

# View result
print(len(interactors_df))

2735


In [39]:
new_edges = pd.DataFrame({
    "Source": protein,
    "Interaction": "pp",  # or another interaction type
    "Target": list(interactors_set)
})

# Append to existing SIF
extended_sif_df = pd.concat([sif_df, new_edges], ignore_index=True)

In [86]:
extended_sif_df.to_csv("/ix/djishnu/Alisa/Tfh/ForPaper/other_figures/extended_vinuesa_hint_ICOS_centric.sif", sep="\t", index=False, header=False)

In [10]:
pps_df  = pd.read_csv('/ix/djishnu/Alisa/Tfh/ForPaper/process_network_prop_output/PPS_1_significant_gene_list.csv')
pps = list(pps_df['Genes'])

In [40]:
# Load Vinuesa gene set
with open("/ix/djishnu/Alisa/Tfh/Vinuesa_updated_list.txt", "r") as file:
    Vinuesa_review = file.read().strip()
Vinuesa_review_gene_set = set(ast.literal_eval(Vinuesa_review))

# Load PPS heat scores
#pps_df = pd.read_csv("/ix/djishnu/Alisa/Tfh/Network_analysis/data/Pattern_1234_Genes_Scores.txt", sep='\t', header=None,)
#pps_df.columns = ["Gene", "PPS_Score"]


#2. Get all unique genes from Source and Target columns
all_genes = list(pd.Series(list(set(extended_sif_df["Source"]).union(set(extended_sif_df["Target"]))), name="Gene"))
#all_genes = list(pd.Series(list(set(sif_df["Source"]).union(set(sif_df["Target"]))), name="Gene"))




In [42]:
gene_presence = []

for gene in all_genes:
    if gene in pps:
        gene_presence.append(1)
    else:
        gene_presence.append(0)

In [43]:
df = pd.DataFrame()
df['name'] = all_genes
df['sig'] = gene_presence

In [44]:
print(df[df['sig']==1])

       name  sig
19   PTPN11    1
60    FOXP1    1
64     IRF4    1
99    CIITA    1
110   RIC8B    1
117   PDCD1    1
125   STAT1    1
128     IL2    1


In [45]:
df.to_csv("/ix/djishnu/Alisa/Tfh/ForPaper/other_figures/extended_vinuesa_hint_CXCR5_sig.csv", index=False)